# Extracción de datos a través de *Beautiful Soap 4*

## 1.- Obtención de datos general de los datos.

En un primer paso vamos a hacer llamadas a través de la librería de BeautifulSoap4 para obtener datos generales desde la primera pagina.

Como siempre lo primero es importar las librerías 

In [2]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [3]:
os.chdir(os.path.split(os.getcwd())[0])
folder=os.getcwd()
folder

'd:\\Data_science\\Javier\\Repositorios\\Proyecto_tienda_online'

### Conectamos la pagina web para acceder a ella y *arañar* los datos de su página web.

In [4]:
url = "https://www.amantis.net/productos-amantis/"              # lista productos
url_principal="https://www.amantis.net/"                        # productos
response = requests.get(url)

In [5]:
html = response.content
soup = bs(html, "lxml")

#### a.- Vamos a obtener información de los datos existente en la URL.

In [6]:
all_h3 = soup.find_all("h3")
all_h3

[<h3 class="t2sDiv-titulo hidden text-left color-corporativo">Top ventas en amantis</h3>,
 <h3 class="h3 group inner list-group-item-heading tdd_listado_nombre">
 <a href="https://www.amantis.net/tobogane-hot-rabbit-el-superventas-amantis-mejorado/">
 <span>TOBOGANE HOT RABBIT, el superventas de amantis ¡mejorado!</span>
 </a>
 </h3>,
 <h3 class="h3 group inner list-group-item-heading tdd_listado_nombre">
 <a href="https://www.amantis.net/ballenato-tu-vibrador-distancia-aleta-movil-sumergible/">
 <span>BALLENATO, tu vibrador a distancia con aleta móvil y sumergible...</span>
 </a>
 </h3>,
 <h3 class="h3 group inner list-group-item-heading tdd_listado_nombre">
 <a href="https://www.amantis.net/tobogane-el-vibrador-doble-mas-vendido-ahora-efecto-hot/">
 <span>TOBOGANE, el vibrador doble más vendido</span>
 </a>
 </h3>,
 <h3 class="h3 group inner list-group-item-heading tdd_listado_nombre">
 <a href="https://www.amantis.net/meneo-sube-baja-realista-control-remoto/">
 <span>MENEO sube y ba

#### b.- Extraemos los nombres desde esta URL.

In [7]:
titulos=soup.find_all("h3")
name=[]
for titulo in titulos[1:]:
    nombre=titulo.get_text(strip=True).split(',')[0]
    name.append(nombre)
print(len(name))
print(name)

24
['TOBOGANE HOT RABBIT', 'BALLENATO', 'TOBOGANE', 'MENEO sube y baja', 'FRESH GIRL', 'LIZO 2', 'REGGIA', 'Pro ANAL', 'TANDEM 2 flex', 'FOXTAIL', 'BISOU', 'TOK2 bala vibradora con mando', 'SAZZIA', 'CRISTALINO XL', 'TRIS-TRAS', 'Vibrador Líquido con sabor Desliz! VIBRAGEL 30ml', 'Kit de 3 Plugs anales con diamante BLACK STAR Plugress', 'IDYLLIC BOY', 'CUORE TRÍO', 'Átame-Fácil', 'YOLA. Suave', 'VULVANIC', 'PASSION PUMP', 'PULSO AIR - Vibrador palpitador con succionador']


#### c.- Con este codigo extraigo la descripción del producto.

In [8]:
titulos=soup.find_all("h3")
desc=[]

for titulo in titulos[1:]:
    description=titulo.get_text(strip=True).split(', ')[1:]
    if description==[]:
        description=["No hay datos"]
    desc.append(description)
print(len(desc))
print(desc)

24
[['el superventas de amantis ¡mejorado!'], ['tu vibrador a distancia con aleta móvil y sumergible...'], ['el vibrador doble más vendido'], ['placer realista con control remoto'], ['6 Kilos y 40cm de piel real disfrutable'], ['Dildo de suave silicona en 3 tamaños'], ['masturbador masculino doble'], ['vibrador anal progresivo'], ['vibrador doble flexible con mando'], ['plug anal cola de zorro de 35cm'], ['besos por ondas de succión con la mejor vibración'], ['más potente y recargable'], ['masturbador hiperrealista'], ['gran dildo transparente de 22cm'], ['anilla con vibrador para doble penetración TRINITY...'], ['hormigueo...'], ['No hay datos'], ['Plug anal a distancia más intenso'], ['Kit de 3 plugs con corazón'], ['kit de cama universal de amantis'], ['Texturizada y Preciosista', 'masturbación definitiva...'], ['Vibrador a distancia ergonómico. Entrarás en erupción...'], ['bomba de succión automática de amantis'], ['No hay datos']]


Vemos que en algunos casos no hay información. 

En otros casos la separación de la descripción, no se ha separado.

Por este motivo, haremos la extracción unicamente de todo y posteriormente lo trataremos con la librería de *pandas*.

#### d.- Con esto extraigo los links de los productos.

In [9]:
productos = soup.find_all(class_='caption')
lista_URLs = []
for producto in productos[8:]:
    URL_producto = producto.find('a')['href']
    lista_URLs.append(URL_producto)

len(lista_URLs)

25

In [16]:
lista_URLs.pop(0)
len(lista_URLs)

24

In [17]:
lista_URLs[0]

'https://www.amantis.net/tobogane-hot-rabbit-el-superventas-amantis-mejorado/'

Esta información es importante ya que nos permitirá obtener las direcciones de cada producto para hacer posteriormente la información desde cada producto.

#### e.- Obtenemos la información de precio rebajado de cada producto.

In [14]:
all_price=soup.find_all("span",class_="productSpecialPrice")
price=[]

for precio in all_price:
    item_price=precio.get_text(strip=True).replace(",", ".").split('€')[0]
    price.append(item_price)
len(price)


24

#### Este es el codigo completo para extraer la información de cada página.

In [43]:
titulos=soup.find_all("h3")
name=[]
desc=[]
price=[]
lista_URLs = []

for titulo in titulos:
    nombre=titulo.get_text(strip=True).split(',')[0]
    name.append(nombre)
    description=titulo.get_text(strip=True).split(', ')[1:]
    if description==[]:
        description=["No hay datos"]
    desc.append(description)

all_price=soup.find_all("span",class_="productSpecialPrice")

for precio in all_price:
    item_price=precio.get_text(strip=True).replace(",", ".").split('€')[0]
    price.append(item_price)

productos = soup.find_all(class_='caption')
for producto in productos[8:]:
    URL_producto = producto.find('a')['href']
    lista_URLs.append(URL_producto)


name.pop(0)
desc.pop(0)
lista_URLs.pop(0)

df_productos = pd.DataFrame({"Name": name,"Description": desc,"Price":price,"link":lista_URLs})
df_productos

,Name,Description,Price,link
0,TOBOGANE HOT RABBIT,[el superventas de amantis ¡mejorado!],39.99,https://www.amantis.net/tobogane-hot-rabbit-el...
1,BALLENATO,[tu vibrador a distancia con aleta móvil y sum...,49.99,https://www.amantis.net/ballenato-tu-vibrador-...
2,TOBOGANE,[el vibrador doble más vendido],36.99,https://www.amantis.net/tobogane-el-vibrador-d...
3,MENEO sube y baja,[placer realista con control remoto],44.99,https://www.amantis.net/meneo-sube-baja-realis...
4,FRESH GIRL,[6 Kilos y 40cm de piel real disfrutable],99.99,https://www.amantis.net/fresh-girl-6-kilos-40c...
5,LIZO 2,[Dildo de suave silicona en 3 tamaños],17.99,https://www.amantis.net/lizo-2-dildo-suave-sil...
6,REGGIA,[masturbador masculino doble],29.99,https://www.amantis.net/reggia-masturbador-mas...
7,Pro ANAL,[vibrador anal progresivo],34.99,https://www.amantis.net/pro-anal-2-vibrador-anal/
8,TANDEM 2 flex,[vibrador doble flexible con mando],59.99,https://www.amantis.net/tandem-2-flex-vibrador...
9,FOXTAIL,[plug anal cola de zorro de 35cm],9.99,https://www.amantis.net/foxtail-plug-anal-cola...


## Vamos a sacar la información de todas las páginas posibles.

Para esto vamos a obtener las URLs, como en el punto d. 

Para extraer los links de los productos y ver si podemos sacar la información de su página concreta.

In [46]:
lista_URLs[0]

'https://www.amantis.net/tobogane-hot-rabbit-el-superventas-amantis-mejorado/'

In [47]:
# Hacemos un nuevo request para el primer libro: 
r = requests.get(lista_URLs[0])

# Creamos una sopa específica con la info de cada libro
soup_producto = bs(r.text, "lxml")

In [ ]:
soup_producto

In [49]:
name = soup_producto.find('h1').text
print(name)

TOBOGANE HOT RABBIT, el superventas de amantis ¡mejorado!


#### a.- Vamos a entrar en una página general para extraer información de ella.

Principalmente vamos a ver como podemos, con un bucle, sacar todos los datos de cada producto.

In [50]:
page=3
url = "https://www.amantis.net/productos-amantis/"              # lista productos
URL = url+'page' + str(page)+'/'
response = requests.get(URL)
titulos=soup.find_all("h3")
titulos

[<h3 class="t2sDiv-titulo hidden text-left color-corporativo">Top ventas en amantis</h3>,
 <h3 class="h3 group inner list-group-item-heading tdd_listado_nombre">
 <a href="https://www.amantis.net/tobogane-hot-rabbit-el-superventas-amantis-mejorado/">
 <span>TOBOGANE HOT RABBIT, el superventas de amantis ¡mejorado!</span>
 </a>
 </h3>,
 <h3 class="h3 group inner list-group-item-heading tdd_listado_nombre">
 <a href="https://www.amantis.net/ballenato-tu-vibrador-distancia-aleta-movil-sumergible/">
 <span>BALLENATO, tu vibrador a distancia con aleta móvil y sumergible...</span>
 </a>
 </h3>,
 <h3 class="h3 group inner list-group-item-heading tdd_listado_nombre">
 <a href="https://www.amantis.net/tobogane-el-vibrador-doble-mas-vendido-ahora-efecto-hot/">
 <span>TOBOGANE, el vibrador doble más vendido</span>
 </a>
 </h3>,
 <h3 class="h3 group inner list-group-item-heading tdd_listado_nombre">
 <a href="https://www.amantis.net/meneo-sube-baja-realista-control-remoto/">
 <span>MENEO sube y ba

Este es el bucle para obtener información anterior de las 5 primeras paginas.

In [74]:
url = "https://www.amantis.net/productos-amantis/"              # lista productos
url_principal="https://www.amantis.net/"                        # productos
response = requests.get(url)


pages= np.arange(2,5)                                   # La primera pagina tiene una serie de datos que no debemos de recoger.
name=[]
desc=[]
price=[]
lista_URLs = []

for page in pages:
    if page == 1:
        print("Pagina",page)
        URL = url
        response = requests.get(url)
        soup = bs(response.text, 'lxml')
        titulos=soup.find_all("h3")

        for titulo in titulos:
            nombre=titulo.get_text(strip=True).split(',')[0]
            name.append(nombre)
            description=titulo.get_text(strip=True).split(', ')[1:]
            if description==[]:
                description=["No hay datos"]
            desc.append(description)
            print(nombre)

        all_price=soup.find_all("span",class_="productSpecialPrice")

        for precio in all_price:
            item_price=precio.get_text(strip=True).replace(",", ".").split('€')[0]
            price.append(item_price)


        productos = soup.find_all(class_='caption')

        for producto in productos[9:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)
            print(URL_producto)
            
            
    else:
        print("Pagina",page)
        URL = url+'page' + str(page)+'/'
        response = requests.get(URL)
        soup = bs(response.text, 'lxml')
        titulos=soup.find_all("h3")

        for titulo in titulos:
            nombre=titulo.get_text(strip=True).split(',')[0]
            name.append(nombre)
            description=titulo.get_text(strip=True).split(', ')[1:]
            if description==[]:
                description=["No hay datos"]
            desc.append(description)
            # print(nombre)

        for precio in all_price:
            item_price=precio.get_text(strip=True).replace(",", ".").split('€')[0]
            price.append(item_price)

        productos = soup.find_all(class_='caption')

        for producto in productos[9:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)
            # print(URL_producto)

print("Nombres: ",len(name))
print("Descripción: ",len(desc))
print("URL: ",len(lista_URLs))
print("Precios: ",len(price))

Pagina 2
Pagina 3
Pagina 4
Nombres:  72
Descripción:  72
URL:  72
Precios:  72


Dado que vemos que hay incongruencias en la longitud del tamaño de cada lista, vamos a realizar un Dataframe unicamente de los datos con la misma longitud.

Esta problemática la solucionaremos más adelante.

In [77]:
df_productos = pd.DataFrame({"Name": name,"Description": desc,"link":lista_URLs})
df_productos.head()

,Name,Description,link
0,MAGARAC,[Kit de arnés + esposas by amantis],https://www.amantis.net/magarac-kit-arnes-espo...
1,PULSO AIR - Vibrador palpitador con succionador,[No hay datos],https://www.amantis.net/pulso-air-vibrador-pal...
2,TYPHOON,[masturbador masculino giroscópico],https://www.amantis.net/typhoon-masturbador-ma...
3,POWER UP METER - Bomba de succión con manómetro,[No hay datos],https://www.amantis.net/power-up-meter-bomba-s...
4,AVENTURE - Vibrador con imán y mando a distancia,[No hay datos],https://www.amantis.net/aventure-vibrador-iman...


#### b.- Verifico que he conseguido las URLs de cada producto, esto es importante para obtener la información de cada producto.

Extraer la información de producto, descripción, enlace y precio tomando los datos desde las URLs de cada producto.

Queda pendiente extraer información de los ratings y los comentarios para establecer un estudio

In [78]:
url = "https://www.amantis.net/productos-amantis/"              # lista productos
url_principal="https://www.amantis.net/"                        # productos
response = requests.get(url)


pages= np.arange(1, 25)
name=[]
desc=[]
price=[]
lista_URLs = []

for page in pages:
    if page == 1:
        print("Pagina",page)
        URL = url
        response = requests.get(url)
        soup = bs(response.text, 'lxml')
        titulos=soup.find_all("h3")

        for titulo in titulos[1:]:
            nombre=titulo.get_text(strip=True).split(',')[0]
            name.append(nombre)
            description=titulo.get_text(strip=True).split(', ')[1:]
            if description==[]:
                description=["No hay datos"]
            desc.append(description)

        all_price=soup.find_all("span",class_="productSpecialPrice")

        for precio in all_price:
            item_price=precio.get_text(strip=True).replace(",", ".").split('€')[0]
            price.append(item_price)


        productos = soup.find_all(class_='caption')

        for producto in productos[9:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)
            
            
    else:
        print("Pagina",page)
        URL = url+'page' + str(page)+'/'
        response = requests.get(URL)
        soup = bs(response.text, 'lxml')
        titulos=soup.find_all("h3")

        for titulo in titulos:
            nombre=titulo.get_text(strip=True).split(',')[0]
            name.append(nombre)
            description=titulo.get_text(strip=True).split(', ')[1:]
            if description==[]:
                description=["No hay datos"]
            desc.append(description)

        for precio in all_price:
            item_price=precio.get_text(strip=True).replace(",", ".").split('€')[0]
            price.append(item_price)

        productos = soup.find_all(class_='caption')

        for producto in productos[9:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)




# df_productos = pd.DataFrame({"Name": name,"Description": desc,"Price":price,"link":lista_URLs})
# df_productos.head()


Pagina 1
Pagina 2
Pagina 3
Pagina 4
Pagina 5
Pagina 6
Pagina 7
Pagina 8
Pagina 9
Pagina 10
Pagina 11
Pagina 12
Pagina 13
Pagina 14
Pagina 15
Pagina 16
Pagina 17
Pagina 18
Pagina 19
Pagina 20
Pagina 21
Pagina 22
Pagina 23
Pagina 24


Verificamos que cantidad de datos tenemos al recorrer todos las paginas con productos.

In [79]:
print("Nombres:" ,len(name))
print("descrip:" ,len(desc))
print("precio:" ,len(price))                    #  Se ve que hay un desajuste en el precio al extraer la información de 1 pagina
print("URL:" ,len(lista_URLs))

Nombres: 576
descrip: 576
precio: 576
URL: 576


In [ ]:
print("Nombres:\n" ,name[:5])
print("descrip:\n" ,desc[:5])
print("precio:\n" ,price[:5])
print("URL:\n" ,lista_URLs[:3])

#### c.- Extrayendo la información de los comentarios de cada producto.

Vamos a entrar en un producto para obtener la información de los comentarios que hay:
- Usuario
- Fecha
- Comentario
- Rating 

In [ ]:
prueba=lista_URLs[1]
response = requests.get(prueba)
soup_prueba = bs(response.text, 'lxml')

titulo=soup_prueba.get_text(strip=True).split(',')[0]
print(titulo)

**Fechas**

Estos datos son en este momento, *string*.

In [ ]:
rating=[]
all_ratings = soup_prueba.find_all("span", class_="date")  
sep_1=(' ')
sep_2=(', ')
# pattern = re.compile(sep)
for ratings in all_ratings:
    rating_coment_1=ratings.get_text(strip=True).split(sep_2)[1]
    rating_coment_2=ratings.get_text(strip=True).split(sep_2)[0]
    rating_coment_3=rating_coment_2.split(sep_1)[1:]
    date_1=rating_coment_3[0]+"/"+rating_coment_3[1]
    date_2=date_1+"/"+rating_coment_1                                       # Estamos pendientes de convertir a fechas, teniendo en cuenta
    # date_object = datetime.strptime(date_2,'%d%m%Y')                        # que esta en español
    # print(type(date_1))
    # print(date_1)
    rating.append(date_2)
    # print(type(date_2))
    # print(date_2)

print(len(rating))
rating

**Usuarios**

In [ ]:
user_comments=[]
all_user_comments = soup_prueba.find_all("span", class_="name-user")  

for user_comment in all_user_comments:
    user_comments.append(user_comment.get_text(strip=True))

print(len(user_comments))
user_comments

**Comentarios**

In [ ]:
comment=[]
all_comments = soup_prueba.find_all("p")
for formats in all_comments[-len(rating):]:                     # Creo que me he cargado 'rating'
    comment.append(formats.get_text(strip=True))

print(comment[1])
len(comment)


Hasta aquí los datos que estamos obteniendo se pasan a una lista por atributo.

### d.- Este es el código bueno para iterar todos los productos extraer la información siguiente:

- Nombre
- Descripción
- Precio sin rebaja (regular_price)
- Precio rebajado (new_price)
- Información de cada producto
- Lista de usuarios que han comentado
- Lista de comentarios
- Lista de Fecha de comentarios
- Lista de Ratings.


In [80]:
url = "https://www.amantis.net/productos-amantis/"              # lista productos
url_principal="https://www.amantis.net/"                        # productos
pages= np.arange(1,5)
# pages= np.arange(1, 25)

'''Listas a generar con la información de los productos'''
lista_URLs = []
name=[]
subname=[]
regular_prices=[]
new_price=[]
# lista_URLs = []
info=[]
charac=[]
user_comments=[]
comment=[]
date=[]
ratings=[]

''' Obtenemos las URLs de los productos para entrar luego en sus URLS y extraer la información'''

for page in pages:
    if page == 1:
        print("Pagina",page)
        URL = url
        response = requests.get(url)
        soup = bs(response.text, 'lxml')
        productos = soup.find_all(class_='caption')
        for producto in productos[9:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)
        
    else:
        print("Pagina",page)
        URL = url+'page' + str(page)+'/'
        response = requests.get(URL)
        soup = bs(response.text, 'lxml')
        productos = soup.find_all(class_='caption')
        for producto in productos[9:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)

'''Extraemos la información de cada producto existente'''

for URL in lista_URLs:
    url_product=URL
    response_product = requests.get(url_product)
    soup_product = bs(response_product.text, 'lxml')
    user_comments_product=[]
    date_comments_product=[]
    comments_product=[]
    rating=[]
    

    titulos=soup_product.find_all("h1",class_="h3")
    for titulo in titulos:
        nombre=titulo.get_text(strip=True).split(',')[0]
        name.append(nombre)
        sub_title=titulo.get_text(strip=True).split(', ')[1:]
        if sub_title==[]:
            sub_title=["No hay datos"]
        subname.append(sub_title)

    all_price = soup_product.find_all("div", class_="productoPrecio pull-right tdd_precio")                        
    for price_container in all_price:                                                                    
        try:
            special_price = price_container.find("span", class_="productSpecialPrice")
            if special_price:
                item_price = float(special_price.get_text(strip=True).replace(",", ".").split('€')[0])
                new_price.append(item_price)
                regular_price = price_container.find("del").get_text(strip=True)
                item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
                regular_prices.append(item_regular_price)
            else:
                regular_price = price_container.find("span").get_text(strip=True)
                item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
                new_price.append(item_regular_price)
                regular_prices.append(None)
        except:
            new_price.append(None)
            regular_prices.append(None)

    description=soup_product.find("div", class_="description") 
    information=description.get_text().split('\n')[1:5]
    info.append(information)
    characteristic=description.get_text().split('\n')[5:-3]
    charac.append(characteristic)
    '''Vamos a obtener los datos de los comentarios de los usuarios'''

    all_user_comments = soup_product.find_all("span", class_="name-user") 
    for user_comment in all_user_comments:
        user_comments_product.append(user_comment.get_text(strip=True))
    user_comments.append(user_comments_product)

    all_dates = soup_product.find_all("span", class_="date")  
    for dates in all_dates:
        dates_text=dates.get_text(strip=True)
        # dates=datetime.strftime(dates, '%dd/%mm/%Y')
        date_comments_product.append(dates_text)
        # date_object = datetime.strptime(date_comments_product)
    date.append(date_comments_product)

    all_comments = soup_product.find_all("p")
    for formats in all_comments[-len(date_comments_product):]:
        comments_product.append(formats.get_text(strip=True))
    comment.append(comments_product)

    hearts = soup_product.find_all('div', class_= 'box-description')
    for heart in hearts:
        heart_rating = heart.find_all('span', class_= 'fas fa-heart')
        num_hearts = len(heart_rating)
        rating.append(num_hearts)
    ratings.append(rating)


for i, regular_price in enumerate(regular_prices):
    if regular_price is None:
        regular_prices[i] = new_price[i]


# print('-'*20)
# print(name)
print('-'*20)
print(len(name))
# print('-'*20)
# print(subname)
print('-'*20)
print(len(subname))
print('-'*20)
# print(regular_prices)
# print('-'*20)
print(len(regular_prices))
# print('-'*20)
# print(new_price)
print('-'*20)
print(len(new_price))
print('-'*20)
print(len(lista_URLs))
# print('-'*20)
# print(lista_URLs)
print('-'*20)
print(len(info))
# print('-'*20)
# # print(charac)
print('-'*20)
print(len(charac))
# print('-'*20)
# # print(user_comments)
print('-'*20)
print(len(user_comments))
# print('-'*20)
# # print(comment)
print('-'*20)
print(len(comment))
# # print('-'*20)
# # print(date)
print('-'*20)
print(len(date))
print('-'*20)
print(len(ratings))

Pagina 1
Pagina 2
Pagina 3
Pagina 4
--------------------
96
--------------------
96
--------------------
96
--------------------
96
--------------------
96
--------------------
96
--------------------
96
--------------------
96
--------------------
96
--------------------
96
--------------------
96


## Visualizamos los datos para ver qué se obtiene y lo guardamos en un fichero .csv.

La intención es depurar el codigo para un mejor manejo de los datos.

In [81]:
dataframe= pd.DataFrame({"Name": name,"Subname": subname,"Description": info, "Characteristics": charac,"Price":regular_price,"Reduced Price":new_price,
                         "date":date,"User": user_comments,"Ratings": ratings,"Comment": comment})
dataframe

,Name,Subname,Description,Characteristics,Price,Reduced Price,date,User,Ratings,Comment
0,TOBOGANE HOT RABBIT,[el superventas de amantis ¡mejorado!],[Vuelve nuestro vibrador de doble estimulación...,"[, , Medidas: 19cm (11cm insertables) y 3,3cm/...",14.99,39.99,"[martes 22 noviembre, 2022, jueves 07 julio, 2...","[Rossi, Marina, Jennifer, Noa, Karen, Lorena, ...","[5, 5, 5, 4, 3, 5, 5, 5, 5, 5, 4, 5, 5, 3, 5, ...","[Mi primera compra. Me encantó la textura, los..."
1,BALLENATO,[tu vibrador a distancia con aleta móvil y sum...,[De las profundidades más húmedas llega BALLEN...,[¡FREE BALLENATO! Ver características y medida...,14.99,49.99,"[sábado 18 febrero, 2023, miércoles 01 febrero...","[Almudena, Tomabel, andrea, Carlos, maria, Mar...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, ...","[Sigo temblando con este juguete, menudas vibr..."
2,TOBOGANE,[el vibrador doble más vendido],"[Por favor, desabróchense los cinturones de se...","[, La última atracción exclusiva de amantis es...",14.99,36.99,"[lunes 06 marzo, 2023, sábado 04 marzo, 2023, ...","[Teresa, Alicia, María, Sara, Raquel, Sara, Da...","[5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, ...",[Fue el primer vibrador que compré. Me lo reco...
3,MENEO sube y baja,[placer realista con control remoto],"[Si te gusta que te metan un buen meneo, hazte...","[Ya lo ves, Meneo’s Cock puede ir contigo a do...",14.99,44.99,"[miércoles 14 diciembre, 2022, miércoles 09 no...","[Francisco, Maria, Jose Javier, Carlos, Pedro,...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4]",[Quería saber cuantos cm tiene la longitud que...
4,FRESH GIRL,[6 Kilos y 40cm de piel real disfrutable],[FRESH GIRL es una bocanada de aire fresco en ...,[FRESH GIRL es tan pesada que tendrás que usar...,14.99,99.99,"[sábado 31 diciembre, 2022, viernes 21 octubre...","[Adrian, Daniel, victor, Guillermo, Jesús, Dav...","[4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[Este juguete es bastante bueno, muy muy place..."
...,...,...,...,...,...,...,...,...,...,...
91,ARMOUR ALAS - Armadura sexual BDSM con alas,[No hay datos],[Las prendas de la cultura BDSM han inundado l...,[Este modelo no es una única pieza si no que e...,14.99,29.99,[],[],[],[Parece que tu navegador está bloqueando JavaS...
92,LAZZO,[Atar tu pene para desatar tu pasión],[Te agarra la base del pene pero para dar rien...,"[Ver características y medidas, , \r, Funda pa...",14.99,12.99,"[sábado 30 abril, 2022, martes 23 noviembre, 2...","[Javier, Miguel]","[4, 4]","[Está hecho de un buen material, es muy elásti..."
93,MYSTERY,[body de rejilla super sexy],[El poder de la seducción está en tus manos co...,"[Siéntente más sexy que nunca, sin renunciar a...",14.99,4.99,"[viernes 09 diciembre, 2022, lunes 28 marzo, 2...","[andrea, María, María, Javier, Lucia, Anonimo]","[4, 3, 5, 4, 5, 5]","[es totalmente precioso, queda estupendo y a m..."
94,Arnés sexual TRIANGLE,[100% Vegano con brillo y sujeción firme],[En los juegos del amarse y disfrutarse todo p...,[Puedes usar este arnés con dildos de base anc...,14.99,12.99,"[sábado 10 diciembre, 2022, jueves 06 octubre,...","[Sara, Sergio, Silvia, Eme, Ramón, Alba, Anne,...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5]",[Es la segunda vez que lo compro y estoy encan...


In [ ]:
file=folder+'\\Data\\scrapped_data.csv'
file

In [ ]:
dataframe.to_csv(file,header=True,index=False)           # Tengo que generar el path correcto


## Solucionando inconvenientes de los datos obtenidos.

- *Subname* puede generar problemas por lo eliminaremos del codigo. Lo generaremos después a partir de *Name*.
- *Characteristics* puede generar problemas, ya que serían datos de dimensiones, duración, etc. Lo generaremos después a partir de *Description*.
- Hay que pasar las listas a registros individuales.

In [ ]:
lista_URLs[0]

In [82]:
url = "https://www.amantis.net/productos-amantis/"              # lista productos
url_principal="https://www.amantis.net/"                        # productos
# pages= np.arange(1,5)
pages= np.arange(1, 25)

'''Listas a generar con la información de los productos'''
lista_URLs = []

''' Obtenemos las URLs de los productos para entrar luego en sus URLS y extraer la información'''

for page in pages:
    if page == 1:
        print("Leyendo paginas")
        # print("Pagina",page)
        URL = url
        response = requests.get(url)
        soup = bs(response.text, 'lxml')
        productos = soup.find_all(class_='caption')
        for producto in productos[9:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)
        
    else:
#        print("Pagina",page)
        URL = url+'page' + str(page)+'/'
        response = requests.get(URL)
        soup = bs(response.text, 'lxml')
        productos = soup.find_all(class_='caption')
        for producto in productos[9:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)
print("Terminando lectura.\nRecabando información.")

list_name=[]
list_regular_prices=[]
list_new_price=[]
list_info=[]
user_comments=[]
comment=[]
date=[]
ratings=[]
datas=[]
diccionario_URL={}



'''Extraemos la información de cada producto existente'''

for URL in lista_URLs:
    url_product=URL
    response_product = requests.get(url_product)
    soup_product = bs(response_product.text, 'lxml')
    user_comments_product=[]
    date_comments_product=[]
    comments_product=[]
    rating=[]
    data=[]
    name=[]
    regular_prices=[]
    new_price=[]
    info=[]

    diccionario_comments={}

    titulos=soup_product.find_all("h1",class_="h3")
    for titulo in titulos:
        nombre=titulo.get_text(strip=True)
        name.append(nombre)

    description=soup_product.find("div", class_="description") 
    information=description.get_text().split('\n')[1:]
    documentation = ''.join(information)
    info.append(documentation)


    all_price = soup_product.find_all("div", class_="productoPrecio pull-right tdd_precio")                        
    for price_container in all_price:                                                                    
        try:
            special_price = price_container.find("span", class_="productSpecialPrice")
            if special_price:
                item_price = float(special_price.get_text(strip=True).replace(",", ".").split('€')[0])
                new_price.append(item_price)
                regular_price = price_container.find("del").get_text(strip=True)
                item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
                regular_prices.append(item_regular_price)
            else:
                regular_price = price_container.find("span").get_text(strip=True)
                item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
                new_price.append(item_regular_price)
                regular_prices.append(None)
        except:
            new_price.append(None)
            regular_prices.append(None)

    for i, regular_price in enumerate(regular_prices):
        if regular_price is None:
            regular_prices[i] = new_price[i]


    '''Vamos a obtener los datos de los comentarios de los usuarios'''
    all_user_comments = soup_product.find_all("span", class_="name-user") 
    for user_comment in all_user_comments:
        user_comments_product.append(user_comment.get_text(strip=True))
    all_dates = soup_product.find_all("span", class_="date")  
    for dates in all_dates:
        dates_text=dates.get_text(strip=True)
        date_comments_product.append(dates_text)
    all_comments = soup_product.find_all("p")
    for formats in all_comments[-len(date_comments_product):]:
        comments_product.append(formats.get_text(strip=True))
    hearts = soup_product.find_all('div', class_= 'box-description')
    for heart in hearts:
        heart_rating = heart.find_all('span', class_= 'fas fa-heart')
        num_hearts = str(len(heart_rating))
        rating.append(num_hearts)
    
    # diccionario={}
    user_comments.append(user_comments_product)
    date.append(date_comments_product)
    comment.append(comments_product)
    ratings.append(rating)
    
    # datos = list(zip(nombre,user_comments_product,comments_product, date_comments_product, rating))
    # for dato in datos:
    #     nombre=dato[0]
    #     comentarios=dato[1:]
    #     if nombre in diccionario_comments:
    #         diccionario_comments[nombre].append(comentarios)
    #     else:
    #         diccionario_comments[nombre] = [comentarios]
    # # diccionario_URL[name] = name
    # diccionario_URL[URL] = diccionario_comments
    
    
# diccionario_URL


dataframe= pd.DataFrame({"Name": name,"Description": info,"Price":regular_price,"Reduced Price":new_price,
                         "date":date,"User": user_comments,"Ratings": ratings,"Comment": comment
                        #  ,"diccionario":diccionario
                        })
dataframe

Leyendo paginas
Terminando lectura.
Recabando información.


ValueError: All arrays must be of the same length

### Realizando las sustituciones pertinentes

In [120]:
url = "https://www.amantis.net/productos-amantis/"              # lista productos
url_principal="https://www.amantis.net/"                        # productos
# pages= np.arange(1,5)
pages= np.arange(1, 25)

'''Listas a generar con la información de los productos'''
lista_URLs = []
name=[]
regular_prices=[]
new_price=[]
info=[]
user_comments=[]
comment=[]
date=[]
ratings=[]
id=[]


''' Obtenemos las URLs de los productos para entrar luego en sus URLS y extraer la información'''

for page in pages:
    if page == 1:
        print("Pagina",page)
        URL = url
        response = requests.get(url)
        soup = bs(response.text, 'lxml')
        productos = soup.find_all(class_='caption')
        for producto in productos[9:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)
        
    else:
        print("Pagina",page)
        URL = url+'page' + str(page)+'/'
        response = requests.get(URL)
        soup = bs(response.text, 'lxml')
        productos = soup.find_all(class_='caption')
        for producto in productos[9:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)


for i in range(len(lista_URLs)):
    id.append(i)

    
'''Extraemos la información de cada producto existente'''

for URL in lista_URLs:
    url_product=URL
    response_product = requests.get(url_product)
    soup_product = bs(response_product.text, 'lxml')
    user_comments_product=[]
    date_comments_product=[]
    comments_product=[]
    rating=[]
    

    titulos=soup_product.find_all("h1",class_="h3")
    for titulo in titulos:
        nombre=titulo.get_text(strip=True)
        name.append(nombre)

    all_price = soup_product.find_all("div", class_="productoPrecio pull-right tdd_precio")                        
    for price_container in all_price:                                                                    
        try:
            special_price = price_container.find("span", class_="productSpecialPrice")
            if special_price:
                item_price = float(special_price.get_text(strip=True).replace(",", ".").split('€')[0])
                new_price.append(item_price)
                regular_price = price_container.find("del").get_text(strip=True)
                item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
                regular_prices.append(item_regular_price)
            else:
                regular_price = price_container.find("span").get_text(strip=True)
                item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
                new_price.append(item_regular_price)
                regular_prices.append(None)
        except:
            new_price.append(None)
            regular_prices.append(None)

    description=soup_product.find("div", class_="description") 
    information=description.get_text().split('\n')[1:]
    documentation = ''.join(information)
    info.append(documentation)


    '''Vamos a obtener los datos de los comentarios de los usuarios'''

    all_user_comments = soup_product.find_all("span", class_="name-user") 
    for user_comment in all_user_comments:
        user_comments_product.append(user_comment.get_text(strip=True))
    user_comments.append(user_comments_product)

    all_dates = soup_product.find_all("span", class_="date")  
    for dates in all_dates:
        dates_text=dates.get_text(strip=True)
        # dates=datetime.strftime(dates, '%dd/%mm/%Y')
        date_comments_product.append(dates_text)
        # date_object = datetime.strptime(date_comments_product)
    date.append(date_comments_product)

    all_comments = soup_product.find_all("p")
    for formats in all_comments[-len(date_comments_product):]:
        comments_product.append(formats.get_text(strip=True))
    comment.append(comments_product)

    hearts = soup_product.find_all('div', class_= 'box-description')
    for heart in hearts:
        heart_rating = heart.find_all('span', class_= 'fas fa-heart')
        num_hearts = len(heart_rating)
        rating.append(num_hearts)
    ratings.append(rating)


for i, regular_price in enumerate(regular_prices):
    if regular_price is None:
        regular_prices[i] = new_price[i]


# print('-'*20)
# print(name)
print('-'*20)
print(len(name))
# print('-'*20)
# print(subname)
# print('-'*20)
# print(len(subname))
print('-'*20)
# print(regular_prices)
# print('-'*20)
print(len(regular_prices))
# print('-'*20)
# print(new_price)
print('-'*20)
print(len(new_price))
print('-'*20)
print(len(lista_URLs))
# print('-'*20)
# print(lista_URLs)
print('-'*20)
print(len(info))
# print('-'*20)
# # print(charac)
# print('-'*20)
# print(len(charac))
# print('-'*20)
# # print(user_comments)
print('-'*20)
print(len(user_comments))
# print('-'*20)
# # print(comment)
print('-'*20)
print(len(comment))
# # print('-'*20)
# # print(date)
print('-'*20)
print(len(date))
print('-'*20)
print(len(ratings))

Pagina 1
Pagina 2
Pagina 3
Pagina 4
Pagina 5
Pagina 6
Pagina 7
Pagina 8
Pagina 9
Pagina 10
Pagina 11
Pagina 12
Pagina 13
Pagina 14
Pagina 15
Pagina 16
Pagina 17
Pagina 18
Pagina 19
Pagina 20
Pagina 21
Pagina 22
Pagina 23
Pagina 24
--------------------
576
--------------------
576
--------------------
576
--------------------
576
--------------------
576
--------------------
576
--------------------
576
--------------------
576
--------------------
576


In [ ]:
lista_URLs

In [112]:

for i in range(len(lista_URLs)):
    id.append(i)
print(id)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [121]:
dataframe= pd.DataFrame({"id":id,"Name": name,"Description": info,"Price":regular_price,"Reduced Price":new_price,
                         "date":date,"User": user_comments,"Ratings": ratings,
                         "Comment": comment})
dataframe

,id,Name,Description,Price,Reduced Price,date,User,Ratings,Comment
0,0,"TOBOGANE HOT RABBIT, el superventas de amantis...",Vuelve nuestro vibrador de doble estimulación ...,39.99,39.99,"[martes 22 noviembre, 2022, jueves 07 julio, 2...","[Rossi, Marina, Jennifer, Noa, Karen, Lorena, ...","[5, 5, 5, 4, 3, 5, 5, 5, 5, 5, 4, 5, 5, 3, 5, ...","[Mi primera compra. Me encantó la textura, los..."
1,1,"BALLENATO, tu vibrador a distancia con aleta m...",De las profundidades más húmedas llega BALLENA...,39.99,49.99,"[sábado 18 febrero, 2023, miércoles 01 febrero...","[Almudena, Tomabel, andrea, Carlos, maria, Mar...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, ...","[Sigo temblando con este juguete, menudas vibr..."
2,2,"TOBOGANE, el vibrador doble más vendido","Por favor, desabróchense los cinturones de seg...",39.99,36.99,"[lunes 06 marzo, 2023, sábado 04 marzo, 2023, ...","[Teresa, Alicia, María, Sara, Raquel, Sara, Da...","[5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, ...",[Fue el primer vibrador que compré. Me lo reco...
3,3,"MENEO sube y baja, placer realista con control...","Si te gusta que te metan un buen meneo, hazte ...",39.99,44.99,"[miércoles 14 diciembre, 2022, miércoles 09 no...","[Francisco, Maria, Jose Javier, Carlos, Pedro,...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4]",[Quería saber cuantos cm tiene la longitud que...
4,4,"FRESH GIRL, 6 Kilos y 40cm de piel real disfru...",FRESH GIRL es una bocanada de aire fresco en t...,39.99,99.99,"[sábado 31 diciembre, 2022, viernes 21 octubre...","[Adrian, Daniel, victor, Guillermo, Jesús, Dav...","[4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[Este juguete es bastante bueno, muy muy place..."
...,...,...,...,...,...,...,...,...,...
571,571,MS-DIGIT. Estimulación Unisex 2 dedos (Cabezal...,La orgásmica colección de accesorios para acop...,39.99,8.99,"[miércoles 30 enero, 2019, miércoles 07 marzo,...","[Anna Maria, Jorge, Natalia, Jose Manuel]","[5, 3, 5, 5]","[Es el cabezal para el Masaje Sexy Pro, nosotr..."
572,572,Collarín BDSM de Suave interior + Cadena con asa,"""... me estremecía la sensación tibia del coll...",39.99,21.99,[],[],[],[Parece que tu navegador está bloqueando JavaS...
573,573,"Anilla con Bolas Anales BLACK LINE, 2 en 1 de ...",¿Creías que era imposible unir en un solo prod...,39.99,17.99,"[jueves 17 septiembre, 2020, viernes 01 mayo, ...","[Ramón, Ricardo]","[2, 5]","[El plug como tal, está bien, pero la anilla n..."
574,574,POMPOM - Plug con pompom,Que se ha practicado sexo anal a lo largo de s...,39.99,14.99,"[sábado 06 marzo, 2021, domingo 08 noviembre, ...","[Maria Elena, Alejandro, Juan Manuel, esther, ...","[5, 5, 5, 5, 5]",[Está muy bien en tema de estética. La única d...


In [117]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 576 entries, 0 to 575
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Name           576 non-null    object 
 1   Description    576 non-null    object 
 2   Price          576 non-null    float64
 3   Reduced Price  576 non-null    float64
 4   date           576 non-null    object 
 5   User           576 non-null    object 
 6   Ratings        576 non-null    object 
 7   Comment        576 non-null    object 
dtypes: float64(2), object(6)
memory usage: 40.5+ KB


In [ ]:
dataframe.to_csv(file,header=True,index=False)           # Tengo que generar el path correcto

In [ ]:
prueba=dataframe[['Name','date','User','Ratings','Comment']]
prueba

In [ ]:
prueba

In [ ]:
lista_URLs[0]

In [ ]:
dataframe.explode('datos')

In [ ]:
dataframe['datos'][0]

In [ ]:
dataframe['user']=dataframe.datos[0]

dataframe['comment']=dataframe.datos[1]
dataframe['date']=dataframe.datos[2]
dataframe['rating']=dataframe.datos[3]

dataframe

